In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms

In [ ]:
img_size = 224

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor()
])

In [ ]:
!git clone https://github.com/anesmeftah/deep-learning-roadmap.git

In [ ]:
trainset = torchvision.datasets.ImageFolder(root='/content/deep-learning-roadmap/Paper Implementation/ViT/test/' , transform=transform)
testset = torchvision.datasets.ImageFolder(root='/content/deep-learning-roadmap/Paper Implementation/ViT/test/' , transform=transform)


In [ ]:
trainset.classes

In [ ]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset , 32 , shuffle= True)
testloader = DataLoader(testset , 32 , shuffle= True)
trainloader , testloader

In [ ]:
image_batch, label_batch = next(iter(trainloader))

image, label = image_batch[0], label_batch[0]

# View the batch shapes
image.shape, label

In [ ]:
# Plot image with matplotlib
plt.imshow(image.permute(1, 2, 0)) # rearrange image dimensions to suit matplotlib [color_channels, height, width] -> [height, width, color_channels]
plt.title(trainset.classes[label])
plt.axis(False)

In [ ]:
# Create example values
height = 224 # H ("The training resolution is 224.")
width = 224 # W
color_channels = 3 # C
patch_size = 16 # P

# Calculate N (number of patches)
number_of_patches = int((height * width) / patch_size**2)
print(f"Number of patches (N) with image height (H={height}), width (W={width}) and patch size (P={patch_size}): {number_of_patches}")

In [ ]:
# Input shape (this is the size of a single image)
embedding_layer_input_shape = (height, width, color_channels)

# Output shape
embedding_layer_output_shape = (number_of_patches, patch_size**2 * color_channels)

print(f"Input shape (single 2D image): {embedding_layer_input_shape}")
print(f"Output shape (single 2D image flattened into patches): {embedding_layer_output_shape}")

In [ ]:
# View single image function
def view_image(trainset , image , label):
  plt.imshow(image.permute(1, 2, 0)) # adjust for matplotlib
  plt.title(trainset.classes[label])
  plt.axis(False)

In [ ]:
view_image(trainset , image , label)

In [ ]:
# Change image shape to be compatible with matplotlib (color_channels, height, width) -> (height, width, color_channels)
image_permuted = image.permute(1, 2, 0)

# Index to plot the top row of patched pixels
patch_size = 16
plt.figure(figsize=(patch_size, patch_size))
plt.imshow(image_permuted[:patch_size, :, :]);

In [ ]:
# Setup hyperparameters and make sure img_size and patch_size are compatible
img_size = 224
patch_size = 16
num_patches = img_size/patch_size
assert img_size % patch_size == 0, "Image size must be divisible by patch size"
print(f"Number of patches per row: {num_patches}\
        \nNumber of patches per column: {num_patches}\
        \nTotal patches: {num_patches*num_patches}\
        \nPatch size: {patch_size} pixels x {patch_size} pixels")

# Create a series of subplots
fig, axs = plt.subplots(nrows=img_size // patch_size, # need int not float
                        ncols=img_size // patch_size,
                        figsize=(num_patches, num_patches),
                        sharex=True,
                        sharey=True)

# Loop through height and width of image
for i, patch_height in enumerate(range(0, img_size, patch_size)): # iterate through height
    for j, patch_width in enumerate(range(0, img_size, patch_size)): # iterate through width

        # Plot the permuted image patch (image_permuted -> (Height, Width, Color Channels))
        axs[i, j].imshow(image_permuted[patch_height:patch_height+patch_size, # iterate through height
                                        patch_width:patch_width+patch_size, # iterate through width
                                        :]) # get all color channels

        # Set up label information, remove the ticks for clarity and set labels to outside
        axs[i, j].set_ylabel(i+1,
                             rotation="horizontal",
                             horizontalalignment="right",
                             verticalalignment="center")
        axs[i, j].set_xlabel(j+1)
        axs[i, j].set_xticks([])
        axs[i, j].set_yticks([])
        axs[i, j].label_outer()

# Set a super title
fig.suptitle(f"{trainset.classes[label]} -> Patchified", fontsize=16)
plt.show()

In [ ]:
patch_size = 16
#Create the Conv2d layer

conv2d = nn.Conv2d(in_channels= 3,
                   out_channels= 768,
                   kernel_size= patch_size,
                   stride= patch_size,
                   padding= 0)


In [ ]:
# View single image

plt.imshow(image.permute(1, 2, 0))
plt.title(trainset.classes[label])
plt.axis(False)

In [ ]:
print(image.shape)
print(image.unsqueeze(0).shape)

In [ ]:
img_out_conv = conv2d(image.unsqueeze(0))
print(img_out_conv.shape)

Visualize five random feature maps to see what they look like

In [ ]:
import random
random_indexes = random.sample(range(0,768) , k=5) #pick 5 numbers
print(f"Showing random convolutional feature maps from indexes: {random_indexes}")

fig , axs = plt.subplots(nrows = 1 , ncols = 5 , figsize=(12,12))
for i, idx in enumerate(random_indexes):
  img_conv_feature_map = img_out_conv[:, idx, : , :]
  axs[i].imshow(img_conv_feature_map.squeeze().detach().numpy())
  axs[i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[]);


Check out the numerical form

In [ ]:
#Get a single feature map in tensor form

single_feature_map = img_out_conv[:, 0, :, :]
single_feature_map , single_feature_map.requires_grad

# Flattening the patch embedding

Reading back the ViT paper it says we don't want to flatten the whole tensor, we only want to flatten the spatial dimension of the feature map which is in our case the `feature_map_height` and the `feature_map_width`

In [ ]:
#Create flatten layer
flatten = nn.Flatten(start_dim = 2, #height
                     end_dim = 3) #width

In [ ]:
# 1. View single image
view_image(trainset , image , label)
print("Original image shape : " , image.shape)

# 2. Turn image into feature maps
img_out_conv = conv2d(image.unsqueeze(0))
print("image feature map shape" , img_out_conv.shape)

# 3. Flatten the feature maps
img_out_conv_flattened = flatten(img_out_conv)
print("Flattened image feature map shape" , img_out_conv_flattened.shape)

In [ ]:
# Get flattened image patch embeddings in right shape
img_out_conv_flattened_reshaped = img_out_conv_flattened.permute(0, 2, 1)
print(f"Patch embedding sequence shape: {img_out_conv_flattened_reshaped.shape} -> [batch_size, num_patches, embedding_size]")

We visualize one of the flattened feature map

In [ ]:
# Get a single flattened feature map
single_flattened_feature_map = img_out_conv_flattened_reshaped[:, :, 0]

# Plot the flattened feature map
plt.figure(figsize=(22,22))
plt.imshow(single_flattened_feature_map.detach().numpy())
plt.title(f"Flattened feature map shape {single_flattened_feature_map.shape}")
plt.axis(False)



> The original Transformer architecture was designed to work with text. The Vision Transformer architecture (ViT) had the goal of using the original Transformer for images. This is why we're taking a 2D image and formatting it so it appears as a 1D sequence of text.



# Turning the ViT patch embedding layer into a PyTorch module

In [ ]:
class PatchEmbedding(nn.Module): # Turns a 2D input image into a 1D sequence learnable embedding vector.

  def __init__(self,
               in_channels = 3,
               patch_size = 16,
               embedding_dim = 768):
    super().__init__()

    self.patcher = nn.Conv2d(in_channels= in_channels,
                             out_channels = embedding_dim,
                             kernel_size = patch_size,
                             stride = patch_size,
                             padding = 0)

    self.flatten = nn.Flatten(start_dim= 2 , end_dim= 3)

  # The forward method
  def forward(self , x):
    image_resolution = x.shape[-1]
    assert image_resolution % patch_size == 0, f"Input image size must be divisible by patch size, image shape: {image_resolution}, patch size: {patch_size}"

    x_patched = self.patcher(x)
    x_flattened = self.flatten(x_patched)

    return x_flattened.permute(0, 2, 1)



In [ ]:
# Create an instance of patch embedding layer
patchify = PatchEmbedding(in_channels=3,
                          patch_size=16,
                          embedding_dim=768)

# Pass a single image through
print(f"Input image shape: {image.unsqueeze(0).shape}")
patch_embedded_image = patchify(image.unsqueeze(0)) # add an extra batch dimension on the 0th index, otherwise will error
print(f"Output patch embedding shape: {patch_embedded_image.shape}")


# Create the class token embedding

In [ ]:
# View the patch embedding and patch embedding shape
print(patch_embedded_image)
print(f"Patch embedding shape: {patch_embedded_image.shape} -> [batch_size, number_of_patches, embedding_dimension]")

In [ ]:
# Get the batch size and embedding dimension
batch_size = patch_embedded_image.shape[0]
embedding_dimension = patch_embedded_image.shape[-1]

class_token = nn.Parameter(torch.ones(batch_size , 1 , embedding_dimension),
                           requires_grad= True)

print(class_token[:, :, :10])
print(f"Class token shape: {class_token.shape} -> [batch_size, number_of_tokens, embedding_dimension]")



In [ ]:
patch_embedded_image_with_class_embedding = torch.cat((class_token , patch_embedded_image) ,
                                                     dim = 1)

# Create the position embedding

In [ ]:
# View the sequence of patch embedding with the prepended class embedding
patch_embedded_image_with_class_embedding , patch_embedded_image_with_class_embedding.shape

In [ ]:
# Calculate N : number of patches
number_patches = int((height * width) / patch_size**2)

# Get embedding dimension
embedding_dimension = patch_embedded_image_with_class_embedding.shape[2]

# Create the learnable 1D position embedding
position_embedding = nn.Parameter(torch.ones(1 , number_patches + 1 , embedding_dimension) , requires_grad= True)

Show the first 10 sequences and 10 position embedding values and check the shape of the position embedding

In [ ]:
print(position_embedding[:, :10, :10])
print(f"Position embedding shape: {position_embedding.shape} -> [batch_size, number_of_patches, embedding_dimension]")

In [ ]:
# Add the position embedding to the patch and class token embedding
patch_and_position_embedding = patch_embedded_image_with_class_embedding + position_embedding
print(patch_and_position_embedding)
print(f"Patch embeddings, class token prepended and positional embeddings added shape: {patch_and_position_embedding.shape} -> [batch_size, number_of_patches, embedding_dimension]")

In [ ]:
# 1. Set patch size
patch_size = 16

# 2. Print shape of original image tensor and get the image dimensions
print(f"Image tensor shape: {image.shape}")
height, width = image.shape[1], image.shape[2]

# 3. Get image tensor and add batch dimension
x = image.unsqueeze(0)
print(f"Input image with batch dimension shape: {x.shape}")

# 4. Create patch embedding layer
patch_embedding_layer = PatchEmbedding(in_channels=3,
                                       patch_size=patch_size,
                                       embedding_dim=768)

# 5. Pass image through patch embedding layer
patch_embedding = patch_embedding_layer(x)
print(f"Patching embedding shape: {patch_embedding.shape}")

# 6. Create class token embedding
batch_size = patch_embedding.shape[0]
embedding_dimension = patch_embedding.shape[-1]
class_token = nn.Parameter(torch.ones(batch_size, 1, embedding_dimension),
                           requires_grad=True)
print(f"Class token embedding shape: {class_token.shape}")

# 7. Prepend class token embedding to patch embedding
patch_embedding_class_token = torch.cat((class_token, patch_embedding), dim=1)
print(f"Patch embedding with class token shape: {patch_embedding_class_token.shape}")

# 8. Create position embedding
number_of_patches = int((height * width) / patch_size**2)
position_embedding = nn.Parameter(torch.ones(1, number_of_patches+1, embedding_dimension),
                                  requires_grad=True)

# 9. Add position embedding to patch embedding with class token
patch_and_position_embedding = patch_embedding_class_token + position_embedding
print(f"Patch and position embedding shape: {patch_and_position_embedding.shape}")

# Multi-Head Attention (MSA)

In [ ]:
class MultiheadSelfAttentionBlock(nn.Module):

  def __init__(self,
               embedding_dim = 768,
               num_heads = 12,
               attn_dropout = 0):
    super().__init__()

    self.layer_norm = nn.LayerNorm(normalized_shape = embedding_dim)

    self.multihead_attn = nn.MultiheadAttention(embed_dim= embedding_dim ,
                                                num_heads= num_heads,
                                                dropout=attn_dropout,
                                                batch_first= True)
  def forward(self , x):
    x = self.layer_norm(x)
    attn_output , _ = self.multihead_attn(query = x,
                                            key = x,
                                            value = x,
                                            need_weights = False)
    return attn_output



We create an instance of our `MultiheadSelfAttentionBlock` and passing through the `patch_and_position_embedding` variable we created

In [ ]:
# Create an instance of MSABlock
multihead_self_attention_block = MultiheadSelfAttentionBlock(embedding_dim=768, # from Table 1
                                                             num_heads=12) # from Table 1

# Pass patch and position image embedding through MSABlock
patched_image_through_msa_block = multihead_self_attention_block(patch_and_position_embedding)
print(f"Input shape of MSA block: {patch_and_position_embedding.shape}")
print(f"Output shape MSA block: {patched_image_through_msa_block.shape}")

# Multilayer Perceptron (MLP)

In [ ]:
class MLPBlock(nn.Module):
  def __init__(self ,
               embedding_dim = 768,
               mlp_size = 3072,
               dropout = 0.1):
    super().__init__()
    self.layer_norm = nn.LayerNorm(normalized_shape= embedding_dim)
    self.mlp = nn.Sequential(
        nn.Linear(in_features= embedding_dim,
                  out_features= mlp_size),
        nn.GELU(),
        nn.Dropout(p = dropout),
        nn.Linear(in_features= mlp_size,
                  out_features= embedding_dim),
        nn.Dropout(p = dropout)
    )

  def forward(self , x):
    x = self.layer_norm(x)
    x = self.mlp(x)
    return x

We create an instance of our `MLPBlock` and passing through the `patched_image_through_msa_block`

In [ ]:
# Create an instance of MLPBlock
mlp_block = MLPBlock(embedding_dim=768,
                     mlp_size=3072,
                     dropout=0.1)

# Pass output of MSABlock through MLPBlock
patched_image_through_mlp_block = mlp_block(patched_image_through_msa_block)
print(f"Input shape of MLP block: {patched_image_through_msa_block.shape}")
print(f"Output shape MLP block: {patched_image_through_mlp_block.shape}")

# Create the Transformer Encoder

In [ ]:
class TransformerEncoderBlock(nn.Module):
  def __init__(self,
               embedding_dim=768,
               num_heads=12,
               mlp_size=3072,
               mlp_dropout=0.1,
               attn_dropout=0):
    super().__init__()
    self.msa_block = MultiheadSelfAttentionBlock(embedding_dim= embedding_dim,
                                                 num_heads= num_heads,
                                                 attn_dropout= attn_dropout)

    self.mlp_block = MLPBlock(embedding_dim= embedding_dim,
                              mlp_size= mlp_size,
                              dropout = mlp_dropout)

  def forward(self, x):
    x = self.msa_block(x) + x
    x = self.mlp_block(x) + x
    return x


# Create ViT

In [ ]:
class ViT(nn.Module):
  def __init__(self,
               img_size = 224,
               in_channels = 3,
               patch_size = 16,
               num_transformer_layers = 12, # layers from Table 1 for ViT-Base
               embedding_dim = 768,
               mlp_size = 3072,
               num_heads = 12,
               attn_dropout = 0,
               mlp_dropout = 0.1,
               embedding_dropout = 0.1,
               num_classes = 1000 # default for ImageNet
               ):
    super().__init__()
    assert img_size % patch_size == 0, f"Image size must be divisible by patch size, image size: {img_size}, patch size: {patch_size}."

    self.num_patches = (img_size ** 2) // patch_size ** 2
    self.class_embedding = nn.Parameter(data = torch.randn(1, 1, embedding_dim),
                                        requires_grad = True)

    self.position_embedding = nn.Parameter(data = torch.randn(1, self.num_patches + 1, embedding_dim),
                                           requires_grad= True)

    self.embedding_dropout = nn.Dropout(p = embedding_dropout)

    self.patch_embedding = PatchEmbedding(in_channels = in_channels,
                                          patch_size = patch_size,
                                          embedding_dim= embedding_dim)

    self.transformer_encoder = nn.Sequential(*[TransformerEncoderBlock(embedding_dim= embedding_dim,
                                                                       num_heads = num_heads,
                                                                       mlp_size = mlp_size,
                                                                       mlp_dropout= mlp_dropout)for _ in range(num_transformer_layers)])

    self.classifier = nn.Sequential(
        nn.LayerNorm(normalized_shape= embedding_dim),
        nn.Linear(in_features= embedding_dim,
                  out_features= num_classes)
    )

  def forward(self, x):

    batch_size = x.shape[0]

    class_token = self.class_embedding.expand(batch_size, -1 ,-1)

    x = self.patch_embedding(x)
    x = torch.cat((class_token, x) , dim = 1)
    x = self.position_embedding + x
    x = self.embedding_dropout(x)
    x = self.transformer_encoder(x)
    x = self.classifier(x[: , 0])

    return x

In [ ]:
# Create a random tensor with same shape as a single image
random_image_tensor = torch.randn(1, 3, 224, 224) # (batch_size, color_channels, height, width)

# Create an instance of ViT with the number of classes we're working with (pizza, steak, sushi)
vit = ViT(num_classes=len(trainset.classes[label]))

# Pass the random image tensor to our ViT instance
vit(random_image_tensor)

# Training

Optimizer : We can see they choose to use "Adam" optimizer
Loss function : we are going to use CrossEntropyLoss because we are working with multi-class classification

In [ ]:
optimizer = torch.optim.Adam(params = vit.parameters(),
                             lr = 0.003,
                             betas= (0.9 , 0.999),
                             weight_decay = 0.3)

loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
# train a model
from tqdm.auto import tqdm

def train(model,
          train_dataloader,
          test_dataloader,
          optimizer,
          loss_fn,
          epochs,
          device = "cuda"):

    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    model.to(device)

    for epoch in range(epochs):
        model.train()
        train_loss, train_correct = 0, 0

        for X, y in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs} [Training]"):
            X, y = X.to(device), y.to(device)

            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            train_correct += (y_pred.argmax(dim=1) == y).sum().item()

        train_loss /= len(train_dataloader)
        train_acc = train_correct / len(train_dataloader.dataset)

        model.eval()
        test_loss, test_correct = 0, 0

        with torch.inference_mode():
            for X, y in tqdm(test_dataloader, desc=f"Epoch {epoch+1}/{epochs} [Testing]"):
                X, y = X.to(device), y.to(device)
                y_pred = model(X)

                test_loss += loss_fn(y_pred, y).item()
                test_correct += (y_pred.argmax(dim=1) == y).sum().item()

        test_loss /= len(test_dataloader)
        test_acc = test_correct / len(test_dataloader.dataset)

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

    return results


In [ ]:
results = train(vit ,
                train_dataloader= trainloader,
                test_dataloader= testloader,
                optimizer= optimizer,
                epochs = 10,
                loss_fn = loss_fn)

In [ ]:
epochs = range(1, len(results["train_loss"]) + 1)

plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(epochs, results["train_loss"], label="Train Loss")
plt.plot(epochs, results["test_loss"], label="Test Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss over Epochs")
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs, results["train_acc"], label="Train Accuracy")
plt.plot(epochs, results["test_acc"], label="Test Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy over Epochs")
plt.legend()

plt.show()
